In [1]:
import tensorflow_datasets as tfds
from sequence_model import DynamicsPredictor, SequenceModel, Encoder
from auxilary_output import Decoder, RewardPredictor, ContinuePredictor
from rlu_unplugged_transform import batch_dataset
import tensorflow as tf

In [12]:
continue_predictor = ContinuePredictor()

out =continue_predictor(tf.zeros((1, 1)))

In [20]:
isinstance()

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [19]:

continue_predictor.binary_crossentropy(tf.zeros((1),dtype=tf.int32), )

TypeError: Cannot convert 0.0 to EagerTensor of dtype int32

In [2]:
dynamics_predictor = DynamicsPredictor()
sequence_model = SequenceModel()
encoder = Encoder()

decoder = Decoder()
reward_predictor = RewardPredictor()
continue_predictor = ContinuePredictor()

In [3]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
kl_divergence = tf.keras.losses.KLDivergence()

recurrent_state = tf.zeros((1, 256))
stochastic_state = tf.zeros((1, 1024))

action = tf.stack([tf.one_hot(0, 16)])
observation = tf.zeros((1, 64, 64, 3))
reward = tf.zeros((1, 1))
continue_flag = tf.zeros((1, 1))

In [4]:
with tf.GradientTape(persistent=True) as tape:
    recurrent_state = sequence_model(recurrent_state=recurrent_state, stochastic_state=stochastic_state, action=action)[0]
    stochastic_dist = encoder.distribution(recurrent_state, observation)
    res = dynamics_predictor.distribution(recurrent_state)

    loss_dynamics = kl_divergence(tf.stop_gradient(stochastic_dist), res)
    loss_representation = kl_divergence(tf.stop_gradient(res), stochastic_dist)
    
    model_state = tf.concat([recurrent_state, stochastic_state], axis=1)
    decoder.loss_fn(model_state, observation)
    # loss_decoder = -tf.math.log(decoder.loss(model_state, observation))
    # loss_reward = -tf.math.log(reward_predictor.loss(model_state, reward))
    # loss_continue = -tf.math.log(continue_predictor.loss(model_state, continue_flag))

    loss = 0.5*loss_dynamics + 0.1*loss_representation + 1 * loss_decoder

grad_var_pairs = []

def _add_grad_var_pair(grad, var):
    grad_var_pairs.extend(list(zip(grad, var)))


for model in [dynamics_predictor, 
              encoder, 
              sequence_model, 
              decoder, 
              reward_predictor, 
              continue_predictor]:
    variables = model.trainable_variables
    grads = tape.gradient(loss, variables)

    _add_grad_var_pair(grads, variables)

optimizer.apply_gradients(grad_var_pairs)





Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


ValueError: in user code:

    File "/Users/sanderhergarten/Documents/programming/Hyppo/src/models/recurrent_world_model/utils.py", line 37, in symlog_loss  *
        return 0.5 * (y_pred - symlog(y_true)) ** 2

    ValueError: Dimensions must be equal, but are 79 and 64 for '{{node sub}} = Sub[T=DT_FLOAT](y_pred, mul)' with input shapes: [1,79,79,192], [1,64,64,3].


In [22]:
data = batch_dataset(tfds.load("rlu_atari", split="train[:5%]"))
k = WorldModel2()
k.compile(optimizer='adam', loss='mse')

2023-03-22 19:14:46.224729: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [23]:
k.fit(data, epochs=1)

: 

: 

In [ ]:

class SequenceModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.recurrent_state_size = config["model_parameters"]["recurrent_state_size"]

        self.mlp_layer_size = config["model_sizes"]["mlp_layer_size"]
        self.gru_recurrent_units = config["model_sizes"]["gru_recurrent_units"]
        self.concat_layer = layers.Concatenate()

        self.gru_layers = [
            layers.GRUCell(units, "relu") for units in self.gru_recurrent_units
        ]

        self.mlp_layers = [
            layers.Dense(layer_size, "relu") for layer_size in self.mlp_layer_size
        ]

        self.output_layer = layers.Dense(self.recurrent_state_size, "sigmoid")

    def call(self, recurrent_state, stochastic_state, action):
        concatinated_inputs = self.concat_layer([stochastic_state, action])

        intermediate_recurrent = concatinated_inputs

        state = recurrent_state

        for layer in self.gru_layers:
            intermediate_recurrent, state = layer(intermediate_recurrent, state)

        intermediate_dense = intermediate_recurrent

        for layer in self.mlp_layers:
            intermediate_dense = layer(intermediate_dense)

        output = self.output_layer(intermediate_dense)

        return output
